In [95]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

import requests

In [96]:
def update_progress(progress):
    bar_length = 50
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

https://de.wikipedia.org/w/api.php?action=parse&format=json

Es sollten am Ende 2.796 Artikel sein

parameter in query:  
list=allcategories https://www.mediawiki.org/wiki/API:Allcategories#Python  
list=allpages https://www.mediawiki.org/wiki/API:Allpages#GET_request
list=categorymembers https://www.mediawiki.org/wiki/API:Categorymembers

https://de.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&format=json&formatversion=2&srsearch=incategory%3AWikipedia%3AExzellent&list=search ES FEHLT NOCH DER GANZE KONTENT, ABER ES SIND QUASI DIE EXZELLENTEN ARTIKEL

&sroffset=9

In [21]:
S = requests.Session()

URL = "https://de.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "rvslots": "*",
    "format": "json",
    "formatversion": 2,
    "srsearch": "incategory:Wikipedia:Exzellent",
    "list": "search",
    "sroffset": 0
}

ids = []

R = S.get(url = URL, params = PARAMS)
DATA = R.json()

for entry in DATA["query"]["search"]:
    ids.append(entry["pageid"])

while DATA.get("continue"):
    PARAMS.update({"sroffset": DATA["continue"]["sroffset"]})
    
    print("\n%s" % (PARAMS))
    R = S.get(url = URL, params = PARAMS)
    DATA = R.json()
    
    for entry in DATA["query"]["search"]:
        ids.append(entry["pageid"])

print(ids)

#if DATA['query']['search'][0]['title'] == SEARCHPAGE:
#    print("Your search page '" + SEARCHPAGE + "' exists on English Wikipedia")


{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 10}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 20}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 30}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 40}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 50}

{'action': 'qu


{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 430}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 440}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 450}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 460}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 470}

{'action'


{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 860}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 870}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 880}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 890}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 900}

{'action'


{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1280}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1290}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1300}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1310}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1320}

{'ac


{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1700}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1710}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1720}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1730}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 1740}

{'ac


{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2120}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2130}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2140}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2150}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2160}

{'ac


{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2540}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2550}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2560}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2570}

{'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'rvslots': '*', 'format': 'json', 'formatversion': 2, 'srsearch': 'incategory:Wikipedia:Exzellent', 'list': 'search', 'sroffset': 2580}

{'ac

In [22]:
len(ids)

2795

https://de.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&pageids=1428&format=json&formatversion=2

In [97]:
PARAMS = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "rvslots": "*",
    "format": "json",
    "formatversion": 2,
    "pageids": 0
}

data = pd.DataFrame()
content = {}

for ix, id in enumerate(ids):
    update_progress(ix / len(ids))
    PARAMS.update({"pageids": id})
    R = S.get(url = URL, params = PARAMS)
    PAGE = R.json()
    content.update({id: PAGE["query"]["pages"][0]["revisions"][0]["slots"]["main"]["content"]})
    #temp_df = pd.DataFrame(PAGE["query"]["pages"][0]["revisions"][0]["slots"])
    #data.drop(["contentformat", "contentmodel"])

Progress: [##################################################] 100.0%


In [55]:
df = pd.DataFrame(PAGE["query"]["pages"][0]["revisions"][0]["slots"])
df

,main
content,{{Dieser Artikel|behandelt das naturwissenscha...
contentformat,text/x-wiki
contentmodel,wikitext


In [98]:
df = pd.DataFrame(content.items())
df = df.rename({0: "pageid", 1:"content"}, axis = 1)

In [99]:
df

,pageid,content
0,1428,"[[Datei:Danimarca XIII secolo, plinio historia..."
1,2677,{{Begriffsklärungshinweis}}\n{{Infobox Staat\n...
2,1200964,{{Dieser Artikel|behandelt den Staat. Zu weite...
3,880316,{{Begriffsklärungshinweis}}\n{{Infobox Staat\n...
4,2391,{{Dieser Artikel|behandelt das Land – zu ander...
...,...,...
2778,11683048,{{Mehrere Bilder\n | Fußzeile = Explosion bei ...
2779,12391297,[[Datei:Canning and Castlereagh.JPG|mini|Canni...
2780,12170024,{{Infobox Kunstwerk\n|bilddatei = \n|bild...
2781,10278319,[[Datei:Sayyid Shaykh al-Hadi bei seiner Reise...


In [100]:
df.to_csv("./2783_excellent_article.csv")